# Find the Top Ten Words of Recall words for each regressor
## "Second" bc this is the second notebook that does this, but in this one, we regress out of the words, not only the other target regressors, but also all of the other stories and schema vectors
### For each story there will be 3 'Top Ten Words', One for each type of regressor
#### 1) Concatenate unique words across all participants
#### 2) For each regressor, regress out of the words the other two vectors. 
#### 3) Correlate the residual wvs with the target regressor. 
#### 4) Index the 10 highest values, and find the 10 highest words
#### 5) Put in dictionary


#### 2/4/21


In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
from random import randrange
from sklearn.metrics import jaccard_score
import random
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
from scipy import stats


from sklearn.decomposition import PCA #for cluster analysis
from gensim.models import KeyedVectors #for word embeddings
import matplotlib.pyplot as plt #for plotting
import os #for importing
import pickle #for loading transcripts
from scipy.stats import pearsonr 

# from _DRAFT_20200604_functions import * #includes constants and score function
from tqdm import tqdm_notebook #for progress bar

%autosave 5

Autosaving every 5 seconds


# 0. Import recalls, and uncentered story and template vectors and sums 

In [4]:
# import recalls and sums
recalls = pickle.load( open( 'fr_recalls', "rb" ) )
sums = pickle.load( open( "fr_sums", "rb" ) )

#import non-centered story, template
templates = pickle.load( open( 'template_vectors', "rb" ) )
stories = pickle.load( open( 'actual_story_vectors', "rb" ) )

In [5]:
top_ten = {}

In [6]:
wikipath = 'rolando/wiki-news-300d-1M.vec'
wv_model = KeyedVectors.load_word2vec_format(wikipath)

In [7]:
wv_dim = 300

# FastText preprocessing, based on bittlingmayer/ft_wiki_preproc.py
# Remove special characters, put spaces between all tokens
SUB = ["s/’/'/g", "s/′/'/g", "s/''/ /g", "s/'/ ' /g", 's/“/"/g', 's/”/"/g', 's/"/ /g', "s/\\./ \\. /g", "s/<br \\/>/ /g", "s/, / , /g", "s/(/ ( /g", "s/)/ ) /g", "s/\\!/ \\! /g", "s/\\?/ \\? /g", "s/\\;/ /g", "s/\\:/ /g", "s/-/ - /g", "s/=/ /g", "s/=/ /g", "s/*/ /g", "s/|/ /g", "s/«/ /g", 
       "s/…/ /g", "s/‘/ /g", "s/í/ /g", "s/ñ/ /g", "s/\x84/ /g", "s/î/ /g", "s/ó/ /g", "s/\x83/ /g", "s/ï/ /g", "s/õ/ /g",
       "s/ò/ /g", "s/,/ /g", "s/ô/ /g", "s/\x92/ /g", "s/é/ /g", "s/\x8e/ /g", "s/â\x80¦/ /g", "s/\x91/ /g", "s/\x93/ /g",
       "s/\x94/ /g", "s/ã®/ /g", "s/ã¨/ /g", "s/ã©/ /g",
       "s/\â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x9d/ /g", "s/â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x98/ /g",
       "s/â/ /g"]

def __normalize_text(s):
    for sg in SUB:
        rep = sg.replace('\\','').split('/')
        s = s.replace(rep[1], rep[2])
    s = s.replace('/',' ')
    return s

def __spaces(s):
    return ' '.join(s.split())

def __digits(s):
    return ''.join(filter(lambda c: not c.isdigit(), s))

# def preproc(s):
#     return __punctuation(__spaces(__digits(__normalize_text(s.lower()))))

def preproc(s):
    return (__spaces(__digits(__normalize_text(s.lower()))))

def __punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

def word2vecSent(sentence, model = 'fasttext'):
    wv_dim = 300 #for glove and fasttext
    
    if model == 'glove':
        wvmodel = glove_model
    elif model == 'fasttext':
        wvmodel = wv_model
        
    words = preproc(sentence).split(' ')
    wv = np.zeros((len(words), wv_dim))
    for i in range(len(words)):
        if words[i] in wvmodel.vocab:
            wv[i,:] = wvmodel.word_vec(words[i])
    
    return words, wv

# I. 3 sets of top ten 
## A. Find unique words in recalls of each story
### 1. Concatenate words and word vectors in parallel across all participants

In [8]:
all_words = {}
all_wvs = {}

#iterate through all recalls in a story and concatente words and wvs of all stories
for key in recalls:
    words = np.zeros((0,1))
    wvs = np.zeros((0,300))
    for i in range(0,3):
        for j in range(0,len(recalls[key][i])):
            p_words, p_wvs = word2vecSent(recalls[key][i][j][0])
            # reshape p_words
            p_words = np.array(p_words)
            p_words = p_words.reshape(p_words.shape[0],-1)
            # stack p_words and p_wvs
            words = np.vstack((words, p_words))
            wvs = np.vstack((wvs, p_wvs))
    all_words[key] = words
    all_wvs[key] = wvs

### 2. Centering all words + story wvs + template wvs

In [9]:
#Concatenate all words from recall

concat = np.zeros((0,300))

for key in all_wvs:
    for i in range(0, len(all_wvs[key])):
        concat = np.vstack((concat, all_wvs[key][i]))
    


In [10]:
print(concat.shape)
for key in stories:
    concat = np.vstack((concat, stories[key]))

(37545, 300)


In [11]:
print(concat.shape)
for key in templates:
    concat = np.vstack((concat, templates[key]))
print(concat.shape)

(37561, 300)
(37569, 300)


In [12]:
centering_vec = np.mean(concat, axis = 0)

# template vectors

for key in templates:
    templates[key] = templates[key] - centering_vec
    
# recall vectors

for key in recalls:
    for i in range(0, len(all_wvs[key])):
        all_wvs[key][i] = all_wvs[key][i] - centering_vec
            
#story vectors

#make new dict with int key instead of string
int_stories = {}
for key in stories:
    int_stories[int(key)] = stories[key] - centering_vec

### 3. Find unique words and wvs

In [13]:
au_words = {}
au_wvs = {}

for key in all_words:
    # find unique wvs and their indices:
    unique_wvs, index = np.unique(all_wvs[key], axis=0, return_index=True)
    unique_words =  all_words[key][index]
    # put in dicts
    au_wvs[key] = unique_wvs
    au_words[key] = unique_words

## B. *Story* Top Ten
### A. Regress out of all the unique_wvs the loc and soc template wvs
#### 1. Concatenating the schema part of the matrix used as input to regression

In [16]:
schema_order = [1, 2, 3, 4, 10, 20, 30, 40]

this_input2 = np.zeros((300,0))

for i in schema_order:
    this_input2 = np.concatenate((this_input2, templates[i].reshape(templates[i].shape[0],-1)), axis = 1)

In [17]:
story_order = [11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44]
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    # add on all other stories to the regression input
    new_input = np.copy(this_input2)
    for story in story_order:
        if story == key:
            print('skip')
        else:
            new_input = np.concatenate((new_input, stories[str(story)].reshape(stories[str(story)].shape[0],-1)), axis = 1)
    print(new_input.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(new_input, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, new_input.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)


### C. Correlate each wv in the recall with the story wv

In [18]:
all_correls = {}

for key in resid_wvs:
    corr = np.zeros((0,1))
    this_story = stories[str(key)].reshape(stories[str(key)].shape[0], -1)
    for i in range(0, len(resid_wvs[key])):
        this_wv = resid_wvs[key][i].reshape(resid_wvs[key][i].shape[0], -1)
        this_correl = np.corrcoef(this_story.T, this_wv.T)
        # add to corr
        corr = np.vstack((corr, this_correl[0,1]))
    all_correls[key] = corr

### D. Top Ten values' indices and words

In [19]:
for key in all_correls:
    print(key)
    this_corr = all_correls[key]
    this_corr = np.ravel(this_corr)
    story_index = this_corr.argsort()[-10:][::-1]
    #print(story_index)
    # index the top words
    top_words = np.zeros((0,1))
    for i in range(0, 10):
        top_words = np.vstack((top_words, au_words[key][story_index[i]]))
    top_words = top_words.ravel()
    print(top_words)
    top_ten[key] = {'story': top_words}

11
['thomas' 'sat' 'got' 't' "'" 'hang' 'burguer' 'hadn' 'ate' 'bar']
33
['ms' 'cheeses' 'stores' 'cheese' 'product' 'products' 'morrison' '.'
 'crackers' 'marketed']
44
['found' 'egyptology' 'mummification' 'saw' 'wrote' 'lived' 'seen' 'knew'
 'during' 'looked']
23
['alex' 'chloe' 'excitement' 'loves' 'likes' 'crowd' 'day' 'everyone'
 'friends' 'mom']
12
['calvin' 'ultimately' 'they' 'conveyor' 'not' 'properly' 'belt' 'was'
 'delay' 'were']
21
['senna' 'oysters' 'dishes' 'caramelized' 'champagne' 'chandeliers'
 'delicious' 'scallops' 'garlic' 'tears']
13
['tonight' 'going' 'think' 'well' 'agreeing' 'trying' 'said' 'however'
 'nodded' 'anyway']
42
['she' 'cart' 'might' 'had' 'her' 'was' 'gave' 'could' 'anna' 'of']
43
['carina' 'simon' 'ribs' 'meat' 'thought' 'chickens' 'blushed' 'met'
 'considered' 'headed']
32
['jeff' 'william' 'wine' 'steep' 'd' '$' 'vineyard' 'per' 'wines'
 'vineyards']
34
['alma' 'fund' 'funds' 'funding' 'investors' 'project' 'investor'
 'finance' 'capital' 'invest

## B. *Location* Top Ten
### 1. Regress out of all the unique_wvs the story and soc template wvs
#### a. Concatenating the story part of the matrix used as input to regression

In [28]:
this_input = np.zeros((300,0))

for story in story_order:
    this_input = np.concatenate((this_input, stories[str(story)].reshape(stories[str(story)].shape[0],-1)), axis = 1)

print(this_input.shape)

(300, 16)


In [31]:
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    loc = key%10
    # Make the inputs of the regression
    new_input = np.copy(this_input)
    for schema in schema_order:
        if schema == loc:
            print('skip')
        else:
            new_input = np.concatenate((new_input, templates[schema].reshape(templates[schema].shape[0],-1)), axis = 1)
    print(new_input.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(new_input, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, new_input.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)


### 2. Correlate each wv in the recall with the story wv

In [32]:
all_correls = {}

for key in resid_wvs:
    corr = np.zeros((0,1))
    loc = key%10
    this_loc = templates[loc].reshape(templates[loc].shape[0], -1)
    for i in range(0, len(resid_wvs[key])):
        this_wv = resid_wvs[key][i].reshape(resid_wvs[key][i].shape[0], -1)
        this_correl = np.corrcoef(this_loc.T, this_wv.T)
        # add to corr
        corr = np.vstack((corr, this_correl[0,1]))
    all_correls[key] = corr


### 3. Top Ten values' indices and words

In [33]:
for key in all_correls:
    print(key)
    this_corr = all_correls[key]
    this_corr = np.ravel(this_corr)
    story_index = this_corr.argsort()[-10:][::-1]
    #print(story_index)
    # index the top words
    top_words = np.zeros((0,1))
    for i in range(0, 10):
        top_words = np.vstack((top_words, au_words[key][story_index[i]]))
    top_words = top_words.ravel()
    print(top_words)
    top_ten[key]['loc'] =  top_words

11
['restaurant' 'food' 'menus' 'restuarant' 'foods' 'menu' 'meal' 'eating'
 'diner' 'restauraunt']
33
['grocery' 'store' 'groceries' 'checkout' 'stores' 'shop' 'supermarket'
 'cashier' 'checkouts' 'wholesale']
44
['lecture' 'class' 'lecturing' 'lectures' 'hall' 'classroom' 'lecturer'
 'attending' 'interrupting' 'school']
23
['grocery' 'store' 'groceries' 'checkout' 'shop' 'grocer' 'supermarket'
 'cashier' 'checkouts' 'buy']
12
['airport' 'airports' 'plane' 'boarding' 'gate' 'flight' 'gates' 'flying'
 'terminal' 'arriving']
21
['restaurant' 'food' 'menus' 'chef' 'menu' 'meal' 'ordering' 'eating'
 'dishes' 'patrons']
13
['grocery' 'store' 'groceries' 'checkout' 'cashier' 'cart' 'buy' 'shelves'
 'shelf' 'shopping']
42
['airport' 'plane' 'planes' 'boarding' 'airplane' 'gate' 'flight'
 'terminal' 'security' 'at']
43
['grocery' 'store' 'groceries' 'checkout' 'shop' 'supermarket' 'cashier'
 'pick' 'shopping' 'counter']
32
['airport' 'plane' 'planes' 'flights' 'airplane' 'gate' 'flight'
 'des

## C. *Social* Top Ten
### 1. Regress out of all the unique_wvs the loc and soc template wvs

In [34]:
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    soc = round(key/10)*10
    # Make the inputs of the regression
    new_input = np.copy(this_input)
    for schema in schema_order:
        if schema == soc:
            print('skip')
        else:
            new_input = np.concatenate((new_input, templates[schema].reshape(templates[schema].shape[0],-1)), axis = 1)
    print(new_input.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(new_input, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, new_input.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)


### 2. Correlate each wv in the recall with the social wv

In [35]:
all_correls = {}

for key in resid_wvs:
    corr = np.zeros((0,1))
    soc = round(key/10)*10
    this_soc = templates[soc].reshape(templates[soc].shape[0], -1)
    for i in range(0, len(resid_wvs[key])):
        this_wv = resid_wvs[key][i].reshape(resid_wvs[key][i].shape[0], -1)
        this_correl = np.corrcoef(this_soc.T, this_wv.T)
        # add to corr
        corr = np.vstack((corr, this_correl[0,1]))
    all_correls[key] = corr

### D. Top Ten values' indices and words

In [36]:
for key in all_correls:
    this_corr = all_correls[key]
    this_corr = np.ravel(this_corr)
    story_index = this_corr.argsort()[-10:][::-1]
    print(story_index)
    # index the top words
    top_words = np.zeros((0,1))
    for i in range(0, 10):
        top_words = np.vstack((top_words, au_words[key][story_index[i]]))
    top_words = top_words.ravel()
    print(top_words)
    top_ten[key]['soc'] = top_words

[471 162 128 211 393 430 233 307  58 297]
['breakup' 'breaking' 'up' 'break' 'breaks' 'wanting' 'for' 'ending'
 'follow' 'considering']
[256 144 457 310 273 471 472  83 442 347]
['deal' 'business' 'deals' 'partnership' 'agreement' 'competitor'
 'handshake' 'work' 'investment' 'profit']
[376 258 220 185 280 248 241 252  94 223]
['the' 'two' 'next' 'other' 'when' 'first' 'couple' 'each' 'what' 'all']
[395 364 489 320 361 381 439 393 504 509]
['ring' 'diamond' 'diamonds' 'proposed' 'proposing' 'proposal' 'been'
 'planning' 'proposes' 'fiance']
[401 106  88 187 150 418 232 452 358 146]
['breakup' 'breaking' 'up' 'split' 'break' 'stated' 'citing' 'departure'
 'wanting' 'expressed']
[362 339 474 447 294 355 394 360 455 461]
['ring' 'diamond' 'wedding' 'diamonds' 'proposed' 'proposal' 'been'
 'planning' 'proposes' 'fiancee']
[487 152 125 319 428 215 403 443 222 496]
['breakup' 'breaking' 'up' 'broke' 'broken' 'break' 'breaks' 'wanting'
 'keeping' 'led']
[385 268 226 187 290 301 259 302 298 35

# II. Presenting Top Ten Words
## A. Export

In [39]:
pickle.dump( top_ten, open( 'top_ten2', "wb" ) )

## B. Print

In [37]:
story_order = [11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44]

for story in story_order:
    print('\n\n',story)
    for keys in top_ten[story]:
        print(keys)
        print(top_ten[story][keys])



 11
story
['thomas' 'sat' 'got' 't' "'" 'hang' 'burguer' 'hadn' 'ate' 'bar']
loc
['restaurant' 'food' 'menus' 'restuarant' 'foods' 'menu' 'meal' 'eating'
 'diner' 'restauraunt']
soc
['breakup' 'breaking' 'up' 'break' 'breaks' 'wanting' 'for' 'ending'
 'follow' 'considering']


 12
story
['calvin' 'ultimately' 'they' 'conveyor' 'not' 'properly' 'belt' 'was'
 'delay' 'were']
loc
['airport' 'airports' 'plane' 'boarding' 'gate' 'flight' 'gates' 'flying'
 'terminal' 'arriving']
soc
['breakup' 'breaking' 'up' 'split' 'break' 'stated' 'citing' 'departure'
 'wanting' 'expressed']


 13
story
['tonight' 'going' 'think' 'well' 'agreeing' 'trying' 'said' 'however'
 'nodded' 'anyway']
loc
['grocery' 'store' 'groceries' 'checkout' 'cashier' 'cart' 'buy' 'shelves'
 'shelf' 'shopping']
soc
['breakup' 'breaking' 'up' 'broke' 'broken' 'break' 'breaks' 'wanting'
 'keeping' 'led']


 14
story
['i' 'maria' 'my' 'her' 'henry' 'me' 'you' 'think' 'she' 'if']
loc
['lecture' 'class' 'lectures' 'teaching' 'ha

In [41]:
top_ten

{11: {'story': array(['hadn', 'could', 'might', 'wasn', 'don', 'can', 'doesn', 'hear',
         'got', 'make'], dtype='<U32'),
  'loc': array(['restaurant', 'food', 'menus', 'menu', 'diner', 'restuarant',
         'foods', 'meal', 'waiter', 'dessert'], dtype='<U32'),
  'soc': array(['breakup', 'breaking', 'for', 'of', 'the', 'break', 'relationship',
         'what', 'considering', 'ending'], dtype='<U32')},
 33: {'story': array(['could', 'doesn', 'wasn', 'can', 'isn', 'should', 'would', 'make',
         'extremely', 'knows'], dtype='<U32'),
  'loc': array(['grocery', 'store', 'groceries', 'stores', 'supermarket',
         'checkout', 'shop', 'checkouts', 'purchase', 'cashier'],
        dtype='<U32'),
  'soc': array(['deal', 'partnership', 'business', 'agreement', 'investment',
         'success', 'the', 'handshake', 'deals', 'competitor'], dtype='<U32')},
 44: {'story': array(['couldn', 'could', 'wasn', 'can', 'doesn', 'hasn', 'don', 'seeing',
         'looked', 'nodded'], dtype='<U32'